
## Bagging与随机森林

### Bagging        
&emsp;&emsp;欲得到泛化性能强的集成,集成中的个体学习器应尽可能相互独立;虽然"独立"在现实任务中无法做到,
但可以设法使基学习器尽可能具有较大的差异.给定一个训练数据集,一种可能的做法是对训练样本进行采样,产生出若干个不同的子集,
再从每个数据子集中训练出一个基学习器.这样，由于训练数据不同，我们获得的基学习器可望具有比较大的差异.然而，为获得好的集成，
我们同时还希望个体学习器不能太差.如果采样出的每个子集都完全不同,则每个基学习器只用到了一小部分训练数据，
甚至不足以进行有效学习，这显然无法确保产生出比较好的基学习器.为解决这个问题，我们可考虑使用相互有交叠的采样子集.        
&emsp;&emsp;先随机取出一个样本放入采样集中,再把该样本放回初始数据集,使得下次采样时该样本仍有可能被选中,这样,经过$m$次随机采样操作,
我们得到包含$m$个样本的采样集,初始训练集中有的样本在采样集里多次出现,有的则从未出现.易知,初始训练集中约有$63.2\%$的样本出现在采样集中.      
&emsp;&emsp;照这样,我们可采样出$T$个包含$m$个训练样本的采样集,然后基于每个采样集训练出一个基学习器,
再将这些基学习器进行结合.这就是Bagging的基本流程.在对预测进行结合时,Bagging通常对分类任务使用简单投票法,
对回归任务使用简单平均法.若分类预测时出现两个类收到同样票数的情形,则最简单的做法是随机选择一个,
也可进一步考察学习器投票的置信度来确定最终胜利者.Bagging的算法描述如下.      

***

输入:训练集$D=\{ (\mathbf{x}_1, y_1) , (\mathbf{x}_2, y_2)(\mathbf{x}_m, y_m)  \}$

&emsp;&emsp;基学习算法$\Phi$;

&emsp;&emsp;训练轮数$T$

过程:

**for** $t=1,2,\cdots,T$ **do**

&emsp;&emsp;$h_t = \Phi(D, \mathcal{D}_{bs})$                    这里$\mathcal{D}_{bs}$是自助采样产生的样本分布

**end for**

输出:$$ H(\mathbf{x}) = \mathrm{arg} \max_{y \in \mathcal{Y}} \sum_{t=1}^T \mathbb{I}(h_t(\mathbf{x}) = y)$$

***

&emsp;&emsp;假定基学习器的计算复杂度为$O(m)$,则Bagging的复杂度大致为$T(O(m) + O(s))$考虑到采样与投票/平均过程的复杂度$O(s)$根小,
而$T$通常是一个不太大的常数,因此,训练一个Bagging 集成与直接使用基学习算法训练一个学习器的复杂度同阶,
这说明Bagging是一个很高效的集成学习算法. 另外,与标准AdaBoost只适用于二分类任务不同(为处理多分类或回归任务,AdaBoost需进行修改),
Bagging能不经修改地用于多分类,回归等任务.        
&emsp;&emsp;值得一提的是,自助采样过程还给Bagging 带来了另一个优点:由于每个基学习器只使用了初始训练集中$63.2\%$ 的样本,
剩下约$36.8\%$的样本可用作验证集来对泛化性能进行"包外估计" (out-of-bag estimate).为此需记录每个基学习器所使用的训练样本.
不妨令$D_t$ 表示$h_t$ 实际使用的训练样本集,令$H^{oob}(\mathbf{x})$表示对样本$\mathbf{x}$ 的包外预测,
即仅考虑那些未使用$\mathbf{x}$训练的基学习器在$\mathbf{x}$上的预测,有     
$$ H^{oob}(\mathbf{x}) = \mathrm{arg} \max_{y \in \mathcal{Y}} \sum_{t=1}^T \mathbb{I}(h_t(\mathbf{x}) = y) \cdot \mathbb{I}(\mathbf{x} \notin D_t) $$      

则Bagging泛化误差的包外估计为   

$$ \epsilon = \frac{1}{|D|} \sum_{(\mathbf{x}, y) \in D} \mathbb{I} ( H^{oob}(\mathbf{x}) \neq y )$$            

&emsp;&emsp;事实上,包外样本还有许多其他用途.例如当基学习器是决策树时,可使用包外样本来辅助剪枝,
或用于估计决策树中各结点的后验概率以辅助对零训练样本结点的处理;当基学习器是是神经网络时,可使用包外样本来辅助早期停止以减小过拟合风险.        


### 随机森林
&emsp;&emsp;随机森林(Random Forest,简称RF) 是Bagging的一个扩展变体.RF在以决策树为基学习器构建Bagging集成的基础上,
进一步在决策树的训练过程中引入了随机属性(即特征)选择.具体来说,传统决策树在选择划分属性时是在当前结点的属性集合(假定有$d$个属性)中选择一个最优属性;
而在RF中,对基决策树的每个结点,先从该结点的属性集合中随机选择一个包含$k$个属性的子集,
然后再从这个子集中选择一个最优属性用于划分. 这里的参数$k $控制了随机性的引入程度;若令$k = d$,则基决策树的构建与传统决策树相同;
若令$k = 1$ ,则是随机选择一个属性用于划分;一般情况下,推荐值$k = \log_2 d$.        
&emsp;&emsp;随机森林简单、容易实现、计算开销小,令人惊奇的是,它在很多现实任务中展现出强大的性能,
被誉为"代表集成学习技术水平的方法".可以看出,随机森林对Bagging 只做了小改动,
但是与Bagging 中基学习器的"多样性"仅通过样本扰动(通过对初始训练集采样)不同的是,随机森林中基学习器的多样性不仅来自样本扰动,
还来自属性扰动,这就使得最终集成的泛化性能可通过个体学习器之间差异度的增加而进一步提升. 